In [1]:
import os
import json
import gc
from pathlib import Path
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
from transformers import AutoTokenizer
from typing import Dict, List, Set, Tuple, NamedTuple, Callable
import scipy
import scml
from scml import pandasx as pdx
from daigt.preprocess import en as pen
tim = scml.Timer()
tim.start()
os.environ["TOKENIZERS_PARALLELISM"] = "false"
percentiles=[.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]
pd.set_option("use_inf_as_na", True)
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
tqdm.pandas()
scml.seed_everything()
info = np.iinfo(np.int16)
print(f"int16, min={info.min}, max={info.max}")

int16, min=-32768, max=32767


In [2]:
min_len, max_len = 773, 32_767  # P01 cutoff

In [3]:
#tokenizer = AutoTokenizer.from_pretrained("huggingface/microsoft/deberta-v3-base", is_fast=True)
#print(f"{repr(tokenizer)}\nmodel_input_names={tokenizer.model_input_names}")

# Combine data sources

In [4]:
edf = pd.read_csv("input/train_essays.csv", low_memory=False)
edf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1378 entries, 0 to 1377
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         1378 non-null   object
 1   prompt_id  1378 non-null   int64 
 2   text       1378 non-null   object
 3   generated  1378 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 43.2+ KB


In [5]:
pdf = pd.read_csv("input/train_prompts.csv", low_memory=False)
pdf = pdf.set_index("prompt_id")
pdf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2 entries, 0 to 1
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   prompt_name   2 non-null      object
 1   instructions  2 non-null      object
 2   source_text   2 non-null      object
dtypes: object(3)
memory usage: 64.0+ bytes


In [6]:
df = pd.read_csv("input/train_drcat_04.csv", low_memory=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44206 entries, 0 to 44205
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   essay_id  44206 non-null  object
 1   text      44206 non-null  object
 2   label     44206 non-null  int64 
 3   source    44206 non-null  object
 4   prompt    12911 non-null  object
 5   fold      44206 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 2.0+ MB


In [7]:
cols = ["essay_id", "text", "label", "source", "prompt"]
df = df[cols]
df = df.rename(columns={"essay_id": "id", "label": "generated"})
df["prompt"] = df["prompt"].fillna("na")

In [8]:
def get_prompt(row)->str:
    p=pdf.loc[row["prompt_id"]]
    name=p["prompt_name"]
    inst=p["instructions"]
    return f"{name} {inst}"
    

edf["prompt"] = edf.progress_apply(get_prompt, axis=1)
edf["source"] = "train_essays"
df = pd.concat([edf, df], ignore_index=True)
cols = ["id", "generated", "source", "prompt", "text"]
df = df[cols].copy()
cols = ["generated"]
df[cols] = df[cols].astype(np.int8)
df.info()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1378/1378 [00:00<00:00, 79436.92it/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45584 entries, 0 to 45583
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         45584 non-null  object
 1   generated  45584 non-null  int8  
 2   source     45584 non-null  object
 3   prompt     45584 non-null  object
 4   text       45584 non-null  object
dtypes: int8(1), object(4)
memory usage: 1.4+ MB


In [9]:
more = len(df)
df = df.drop_duplicates(["text"], ignore_index=True)
print(f"{more - len(df)} rows dropped: duplicates")

1429 rows dropped: duplicates


# Preprocess Text

In [10]:
bsc = pen.BasicPreprocessor()
bow = pen.BowPreprocessor()

def preprocess_bsc(col) -> Callable:
    def fn(row) -> str:
        return bsc(row[col])
    
    return fn

def preprocess_bow(col) -> Callable:
    def fn(row) -> str:
        return bow(row[col], drop_stopword=False)
    
    return fn

In [11]:
cols = ["text", "prompt"]
for col in cols:
    print(col)
    new_col = f"{col}_bsc" 
    df[new_col] = df.progress_apply(preprocess_bsc(col), axis=1)
    new_col = f"{col}_bow" 
    df[new_col] = df.progress_apply(preprocess_bow(col), axis=1)
    col1, col2 = new_col, f"{col}_bow_len"
    df[col2] = df[col1].str.len()
    df[col2] = df[col2].astype(np.int16)
    #x = tokenizer(df[new_col].tolist(), truncation=False, add_special_tokens=False)
    #len_col = f"{new_col}_len" 
    #df[len_col] = [len(s) for s in x["input_ids"]]
    #df[len_col] = df[len_col].astype(np.int16) 

text


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 44155/44155 [18:21<00:00, 40.10it/s]


prompt


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 44155/44155 [00:56<00:00, 781.58it/s]


# Filter Invalid Data

In [12]:
more = len(df)
df = df.drop_duplicates(["text_bow"], ignore_index=True)
print(f"{more - len(df)} rows dropped: duplicates")

221 rows dropped: duplicates


In [13]:
df.describe(percentiles=percentiles)

,generated,text_bow_len,prompt_bow_len
count,43934.000000,43934.000000,43934.000000
mean,0.328038,2166.630423,103.456275
std,0.469504,964.377584,169.065939
min,0.000000,234.000000,2.000000
1%,0.000000,782.330000,2.000000
5%,0.000000,966.000000,2.000000
10%,0.000000,1127.000000,2.000000
20%,0.000000,1393.000000,2.000000
30%,0.000000,1609.000000,2.000000
40%,0.000000,1806.000000,2.000000


In [14]:
more = len(df)
df = df[(df["text_bow_len"]>=min_len) & (df["text_bow_len"]<=max_len)].copy()
print(f"{more - len(df)} rows dropped: text length must be between {min_len} and {max_len}")

405 rows dropped: duplicates


# Review Data

In [15]:
df = df.reset_index(drop=True) 
df = df.drop(columns=["id"])
df["essay_id"] = df.index
df["essay_id"] = df["essay_id"].astype(np.int32)
cols = list(df.columns)
cols = [cols[-1]] + cols[:-1]
df = df[cols]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43529 entries, 0 to 43528
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   essay_id        43529 non-null  int32 
 1   generated       43529 non-null  int8  
 2   source          43529 non-null  object
 3   prompt          43529 non-null  object
 4   text            43529 non-null  object
 5   text_bsc        43529 non-null  object
 6   text_bow        43529 non-null  object
 7   text_bow_len    43529 non-null  int16 
 8   prompt_bsc      43529 non-null  object
 9   prompt_bow      43529 non-null  object
 10  prompt_bow_len  43529 non-null  int16 
dtypes: int16(2), int32(1), int8(1), object(7)
memory usage: 2.7+ MB


In [16]:
df.describe(percentiles=percentiles)

,essay_id,generated,text_bow_len,prompt_bow_len
count,43529.000000,43529.000000,43529.000000,43529.000000
mean,21764.000000,0.323853,2180.673321,102.230743
std,12565.884271,0.467950,957.700396,168.787156
min,0.000000,0.000000,773.000000,2.000000
1%,435.280000,0.000000,841.000000,2.000000
5%,2176.400000,0.000000,996.000000,2.000000
10%,4352.800000,0.000000,1152.000000,2.000000
20%,8705.600000,0.000000,1409.000000,2.000000
30%,13058.400000,0.000000,1622.000000,2.000000
40%,17411.200000,0.000000,1816.000000,2.000000


In [17]:
cols=["generated"]
pdx.value_counts(df[cols])

,count,percent
generated,,
0,29432,0.676147
1,14097,0.323853


In [18]:
cols=["prompt_bow"]
pdx.value_counts(df[cols]).head(50)

,count,percent
prompt_bow,,
na,29713,0.682602
car free cities write an explanatory essay to inform fellow citizens about the advantages of limiting car usage your essay must be based on ideas and information that can be found in the passage set manage your time carefully so that you can read the passages plan your response write your response and revise and edit your response be sure to use evidence from multiple sources and avoid overly relying on one source your response should be in the form of a multiparagraph essay write your essay in the space provided,708,0.016265
does the electoral college work write a letter to your state senator in which you argue in favor of keeping the electoral college or changing to election by popular vote for the president of the united states use the information from the texts in your essay manage your time carefully so that you can read the passages plan your response write your response and revise and edit your response be sure to include a claim address counterclaims use evidence from multiple sources and avoid overly relying on one source your response should be in the form of a multiparagraph essay write your response in the space provided,670,0.015392
when people ask for advice they sometimes talk to more than one person explain why seeking multiple opinions can help someone make a better choice use specific details and examples in your response,144,0.003308
some schools offer distance learning as an option for students to attend classes from home by way of online or video conferencing do you think students would benefit from being able to attend classes from home take a position on this issue support your response with reasons and examples,143,0.003285
your principal has decided that all students must participate in at least one extracurricular activity for example students could participate in sports work on the yearbook or serve on the student council do you agree or disagree with this decision use specific details and examples to convince others to support your position,142,0.003262
in the article making mona lisa smile the author describes how a new technology called the facial action coding system enables computers to identify human emotions using details from the article write an essay arguing whether the use of this technology to read the emotional expressions of students in a classroom is valuable,139,0.003193
you have read the article unmasking the face on mars imagine you are a scientist at nasa discussing the face with someone who thinks it was created by aliens using information in the article write an argumentative essay to convince someone that the face is just a natural landform be sure to include claims to support your argument that the face is a natural landform evidence from the article to support your claims an introduction a body and a conclusion to your argumentative essay,135,0.003101
today the majority of humans own and operate cell phones on a daily basis in essay form explain if drivers should or should not be able to use cell phones in any capacity while operating a vehicle,134,0.003078


In [19]:
cols=["source"]
pdx.value_counts(df[cols])

,count,percent
source,,
persuade_corpus,25793,0.592547
mistral7binstruct_v1,2420,0.055595
llama2_chat,2418,0.055549
mistral7binstruct_v2,2409,0.055342
original_moth,2264,0.052011
chat_gpt_moth,2124,0.048795
train_essays,1378,0.031657
llama_70b_v1,1172,0.026925
falcon_180b_v1,1051,0.024145


In [20]:
df.groupby(["source", "generated"])["essay_id"].count()

source                generated
chat_gpt_moth         1             2124
darragh_claude_v6     1             1000
darragh_claude_v7     1             1000
falcon_180b_v1        1             1051
llama2_chat           1             2418
llama_70b_v1          1             1172
mistral7binstruct_v1  1             2420
mistral7binstruct_v2  1             2409
original_moth         0             2264
persuade_corpus       0            25793
radek_500             1              500
train_essays          0             1375
                      1                3
Name: essay_id, dtype: int64

In [21]:
df.sample(5).head()

essay_id  generated           source  \
18893     18893          0  persuade_corpus   
12302     12302          1   falcon_180b_v1   
2242       2242          0  persuade_corpus   
32044     32044          0  persuade_corpus   
39574     39574          1      llama2_chat   

                                                                                                                                                                                                                                                                   prompt  \
18893                                                                                                                                                                                                                                                                  na   
12302  The use of standardized testing in schools has been a topic of debate for many years. Write an essay stating your position on whether standardized tests are an accurate measure of student ability, using specific reasons and examples to support your argument.   
2242                                                                                                                                                                                                                                                                   na   
32044                                                                                                                                                                                                                                                                  na   
39574                                                           Task: Evaluate the potential pros and cons of three year high school graduation programs for high school students and analyze how this option would affect academics, extracurriculars, and student life.   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

In [22]:
%%time
df.to_parquet(f"output/preprocess.parquet", index=False)
assert df.notna().all(axis=None)

CPU times: user 603 ms, sys: 64 ms, total: 667 ms
Wall time: 676 ms


In [23]:
tim.stop()
print(f"Total time taken {str(tim.elapsed)}")

Total time taken 0:19:32.812609
